### GRU
GRU将LSTM简化为两个门，重置门和更新门，复杂度大大降低。从效果来看，GRU通常优于LSTM

![gru_image](../images/gru_image.png)

In [2]:
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
from    keras import layers


tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

In [3]:
batchsz = 128

# the most frequest words
total_words = 10000
max_review_len = 80
embedding_len = 100
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=total_words)
# x_train:[b, 80]
# x_test: [b, 80]
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_review_len)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=max_review_len)

db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.shuffle(1000).batch(batchsz, drop_remainder=True)
db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.batch(batchsz, drop_remainder=True)
print('x_train shape:', x_train.shape, tf.reduce_max(y_train), tf.reduce_min(y_train))
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80) tf.Tensor(1, shape=(), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
x_test shape: (25000, 80)


In [4]:
class MyGRUByCell(keras.Model):

    def __init__(self, units):
        super(MyGRUByCell, self).__init__()

        # [b, 64]
        self.state0 = [tf.zeros([batchsz, units])]
        self.state1 = [tf.zeros([batchsz, units])]

        # transform text to embedding representation
        # [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)

        # [b, 80, 100] , h_dim: 64
        # RNN: cell1 ,cell2, cell3
        # SimpleRNN
        # self.rnn_cell0 = layers.SimpleRNNCell(units, dropout=0.5)
        # self.rnn_cell1 = layers.SimpleRNNCell(units, dropout=0.5)
        self.rnn_cell0 = layers.GRUCell(units, dropout=0.5)
        self.rnn_cell1 = layers.GRUCell(units, dropout=0.5)


        # fc, [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = layers.Dense(1)

    def call(self, inputs, training=None):
        """
        net(x) net(x, training=True) :train mode
        net(x, training=False): test
        :param inputs: [b, 80]
        :param training:
        :return:
        """
        # [b, 80]
        x = inputs
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute
        # [b, 80, 100] => [b, 64]
        state0 = self.state0
        state1 = self.state1
        for word in tf.unstack(x, axis=1): # word: [b, 100]
            # h1 = x*wxh+h0*whh
            # out0: [b, 64]
            out0, state0 = self.rnn_cell0(word, state0, training)
            # out1: [b, 64]
            out1, state1 = self.rnn_cell1(out0, state1, training)

        # out: [b, 64] => [b, 1]
        x = self.outlayer(out1)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

def cell_main():
    units = 64
    epochs = 4

    import time

    t0 = time.time()

    model = MyGRUByCell(units)
    model.compile(optimizer = keras.optimizers.Adam(0.001),
                  loss = tf.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])
    model.fit(db_train, epochs=epochs, validation_data=db_test)

    model.evaluate(db_test)

    t1 = time.time()
    # LSTM: 64.3 seconds, 83.4%
    # GRU:  96.7s, 83.4%
    print('total time cost:', t1-t0)

In [5]:
class MyGRUByLayer(keras.Model):

    def __init__(self, units):
        super(MyGRUByLayer, self).__init__()


        # transform text to embedding representation
        # [b, 80] => [b, 80, 100]
        self.embedding = layers.Embedding(total_words, embedding_len,
                                          input_length=max_review_len)

        # [b, 80, 100] , h_dim: 64
        self.rnn = keras.Sequential([
            # layers.SimpleRNN(units, dropout=0.5, return_sequences=True, unroll=True),
            # layers.SimpleRNN(units, dropout=0.5, unroll=True)

            # unroll: Boolean (default False). If True, the network will be unrolled,
            # else a symbolic loop will be used.
            # Unrolling can speed-up a RNN, although it tends to be more memory-intensive.
            # Unrolling is only suitable for short sequences.
            layers.GRU(units, dropout=0.5, return_sequences=True),
            layers.GRU(units, dropout=0.5)
        ])


        # fc, [b, 80, 100] => [b, 64] => [b, 1]
        self.outlayer = layers.Dense(1)

    def call(self, inputs, training=None):
        """
        net(x) net(x, training=True) :train mode
        net(x, training=False): test
        :param inputs: [b, 80]
        :param training:
        :return:
        """
        # [b, 80]
        x = inputs
        # embedding: [b, 80] => [b, 80, 100]
        x = self.embedding(x)
        # rnn cell compute
        # x: [b, 80, 100] => [b, 64]
        x = self.rnn(x)

        # out: [b, 64] => [b, 1]
        x = self.outlayer(x)
        # p(y is pos|x)
        prob = tf.sigmoid(x)

        return prob

def layer_main():
    units = 64
    epochs = 4

    import time


    t0 = time.time()
    model = MyGRUByLayer(units)
    model.compile(optimizer = keras.optimizers.Adam(0.001),
                  loss = tf.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

    model.fit(db_train, epochs=epochs, validation_data=db_test)

    model.evaluate(db_test)


    t1 = time.time()
    # Unroll=True
    # LSTM: 69.3 secnods, 83%
    # GRU: 100 seconds, 83.4%

    # Unroll=False
    # LSTM：23.71， 81.24
    # GRU 23.05， 83.11
    # 这里效率主要和cudann相关，模型的参数需要符合cudann的要求
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU#used-in-the-notebooks
    print('total time cost:', t1-t0)

In [6]:
cell_main()

Epoch 1/4
195/195 [==============================] - 48s 163ms/step - loss: 0.5174 - accuracy: 0.7282 - val_loss: 0.3841 - val_accuracy: 0.8368
Epoch 2/4
195/195 [==============================] - 35s 179ms/step - loss: 0.3307 - accuracy: 0.8598 - val_loss: 0.3762 - val_accuracy: 0.8376
Epoch 3/4
195/195 [==============================] - 38s 194ms/step - loss: 0.2705 - accuracy: 0.8908 - val_loss: 0.3870 - val_accuracy: 0.8372
Epoch 4/4
195/195 [==============================] - 9s 45ms/step - loss: 0.4145 - accuracy: 0.8306
total time cost: 167.94335746765137


In [7]:
layer_main()

Epoch 1/4
195/195 [==============================] - 48s 233ms/step - loss: 0.4984 - accuracy: 0.7386 - val_loss: 0.3633 - val_accuracy: 0.8363
Epoch 2/4
195/195 [==============================] - 47s 242ms/step - loss: 0.3077 - accuracy: 0.8720 - val_loss: 0.3721 - val_accuracy: 0.8383
Epoch 3/4
195/195 [==============================] - 47s 240ms/step - loss: 0.2511 - accuracy: 0.8989 - val_loss: 0.3982 - val_accuracy: 0.8354
Epoch 4/4
195/195 [==============================] - 11s 59ms/step - loss: 0.4392 - accuracy: 0.8247
total time cost: 200.74364137649536
